<a href="https://colab.research.google.com/github/harshithamadarapu/Team16_Hinglish-Auto-suggestions/blob/main/training%20using%20distill%20bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

# Load word embeddings
word_embeddings_path = '/content/vectors.csv'
word_embeddings_df = pd.read_csv(word_embeddings_path)

# Create a dictionary mapping words to their embeddings
word_to_embedding = {row['word']: row[1:].values.astype(np.float32) for _, row in word_embeddings_df.iterrows()}
print("Loaded word embeddings for", len(word_to_embedding), "words.")


Loaded word embeddings for 31887 words.


In [ ]:
# Load training and validation datasets
train_data_path = '/content/train_data.csv'
val_data_path = '/content/val_data.csv'

train_df = pd.read_csv(train_data_path)
val_df = pd.read_csv(val_data_path)

# Display first few rows of the train and validation datasets
print("Training Data:")
print(train_df.head())
print("Validation Data:")
print(val_df.head())


Training Data:
                                             phrases
0                 morning alarm daily ko repeat kare
1  Muje the Charger ' s game me 8 pm tak pohochne...
2        Lakewood , Ohio ka weekly weather kya hai ?
3            mujhe temperature badhne par alert karo
4             Aaj raat Miami me karne ke liye cheeze
Validation Data:
                                             phrases
0  kya S Ã£ o Paulo Avenue se Beyonce show me jan...
1             kya lightning aane ka koi chance hai ?
2  Mike ko message kare ki mai kal 5 am ko meetin...
3  true story ,  teachers staff room main bina te...
4                     1 am ke liye alarm create kare


In [ ]:
import re

# Basic cleaning function
def clean_text(text):
    # Convert to lowercase (if using uncased model)
    text = text.lower()

    # Remove non-alphabetic characters (except for spaces)
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Keep only alphabetic characters and spaces

    # Remove alphanumeric words (words that contain both letters and numbers)
    text = re.sub(r'\b\w*\d\w*\b', '', text)  # Remove words that contain numbers

    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Apply cleaning function to train and validation data and update the "phrases" column
train_df['phrases'] = train_df['phrases'].apply(clean_text)
val_df['phrases'] = val_df['phrases'].apply(clean_text)

# Verify the changes
print(train_df['phrases'].head())
print(val_df['phrases'].head())


0                   morning alarm daily ko repeat kare
1    muje the charger s game me pm tak pohochne ke ...
2              lakewood ohio ka weekly weather kya hai
3              mujhe temperature badhne par alert karo
4               aaj raat miami me karne ke liye cheeze
Name: phrases, dtype: object
0    kya s o paulo avenue se beyonce show me jane k...
1                 kya lightning aane ka koi chance hai
2    mike ko message kare ki mai kal am ko meeting ...
3    true story teachers staff room main bina teach...
4                         am ke liye alarm create kare
Name: phrases, dtype: object


In [ ]:
from transformers import DistilBertTokenizer

# Load the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenizing function
def tokenize_data(dataframe):
    return tokenizer(
        dataframe['phrases'].tolist(),
        padding=True,
        truncation=True,
        max_length=128,  # You can adjust max length based on your data
        return_tensors='pt'
    )

# Apply tokenization
train_tokens = tokenize_data(train_df)
val_tokens = tokenize_data(val_df)


In [ ]:
from torch.utils.data import Dataset

class HinglishDataset(Dataset):
    def __init__(self, tokenized_data):
        self.input_ids = tokenized_data['input_ids']
        self.attention_mask = tokenized_data['attention_mask']
        self.labels = tokenized_data['input_ids']  # For MLM task, labels are same as inputs

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx]
        }

# Create datasets for training and validation
train_dataset = HinglishDataset(train_tokens)
val_dataset = HinglishDataset(val_tokens)


In [ ]:
from transformers import DistilBertForMaskedLM, TrainingArguments, Trainer

# Load DistilBERT model for masked language modeling
model = DistilBertForMaskedLM.from_pretrained('distilbert-base-uncased')

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',              # Output directory
    evaluation_strategy="epoch",         # Evaluation strategy
    learning_rate=2e-5,                  # Learning rate
    per_device_train_batch_size=8,       # Batch size for training
    per_device_eval_batch_size=8,        # Batch size for evaluation
    num_train_epochs=3,                  # Number of training epochs
    weight_decay=0.01,                   # Weight decay
    logging_dir='./logs',                # Logging directory
    fp16=True,                           # Mixed precision for faster training
)

# Initialize the Trainer
trainer = Trainer(
    model=model,                         # The model to be trained
    args=training_args,                  # Training arguments
    train_dataset=train_dataset,         # Training dataset
    eval_dataset=val_dataset             # Validation dataset
)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
# Start the training
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


TrainOutput(global_step=46089, training_loss=0.007263169806896317, metrics={'train_runtime': 4570.0719, 'train_samples_per_second': 80.676, 'train_steps_per_second': 10.085, 'total_flos': 1.2218725142281728e+16, 'train_loss': 0.007263169806896317, 'epoch': 3.0})

In [ ]:
# Evaluate the model on the validation dataset
results = trainer.evaluate()

# Print evaluation results
print("Evaluation results:", results)


Evaluation results: {'eval_loss': 2.798308429419194e-08, 'eval_runtime': 32.6207, 'eval_samples_per_second': 470.928, 'eval_steps_per_second': 58.889, 'epoch': 3.0}


In [ ]:
# Save the model and tokenizer
model.save_pretrained('/content/hinglish_auto_suggestion_model')
tokenizer.save_pretrained('/content/hinglish_auto_suggestion_tokenizer')


('/content/hinglish_auto_suggestion_tokenizer/tokenizer_config.json',
 '/content/hinglish_auto_suggestion_tokenizer/special_tokens_map.json',
 '/content/hinglish_auto_suggestion_tokenizer/vocab.txt',
 '/content/hinglish_auto_suggestion_tokenizer/added_tokens.json')

In [ ]:
# Load the trained model and tokenizer
model = DistilBertForMaskedLM.from_pretrained('/content/hinglish_auto_suggestion_model')
tokenizer = DistilBertTokenizer.from_pretrained('/content/hinglish_auto_suggestion_tokenizer')

# Function to get next word prediction
def get_next_word(text, mask="[MASK]"):
    input_text = text + " " + mask
    inputs = tokenizer(input_text, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # Get the predicted word id for the [MASK] token
    mask_token_index = torch.where(inputs.input_ids == tokenizer.mask_token_id)[1]
    predicted_token_id = logits[0, mask_token_index, :].argmax(dim=-1)

    predicted_word = tokenizer.decode(predicted_token_id)
    return predicted_word

# Example usage
text = "The weather is so"
predicted_word = get_next_word(text)
print(f"Predicted next word: {predicted_word}")


Predicted next word: [PAD]
